Placeholder for development and debugging

User story -> we provide a file_path, if valid file the image is processed and the result of processing are the attributes of the entity

In [1]:
import imghdr
import requests
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
## Const
HTTP_OK = 200
HTTP_BAD_REQUEST = 400
HTTP_UNAUTHORIZED = 401

## Configurable
IP_ADDRESS = 'localhost'
PORT = '5000'

In [3]:
def get_matched_faces(predictions : dict):
    """
    Get the predicted faces and their confidence.
    """
    try:
        matched_faces = {face['userid']: round(face['confidence']*100, 1) 
            for face in predictions if not face['userid'] == 'unknown'}
        return matched_faces
    except:
        return {}

In [4]:
def is_valid_image(file_path : str):
    """
    Check file_path is valid image.
    """
    try:
        image_extension = imghdr.what(file_path)
        if image_extension in ['jpeg', '.jpg', '.png']:
            return True
        return False
    except Exception as exc:
        print(exc)
        return False

In [5]:
def post_image(url : str, image : bytes):
    """Post an image to the classifier."""
    try:
        response = requests.post(
            url,
            files={"image": image},
            )
        return response
    except requests.exceptions.ConnectionError:
        print("ConnectionError: Is %s running?", CLASSIFIER)
        return None

Create classes

In [6]:
class DeepstackFace():
    """Work with faces."""

    def __init__(self, ip_address, port):

        self._url_check = "http://{}:{}/v1/vision/face/recognize".format(
            ip_address, port)
        self._faces = None
        self._matched = {}

    def process_image_bytes(self, image_bytes):
        """Process an image."""
        response = post_image(
            self._url_check, image_bytes)
        if response:
            if response.status_code == HTTP_OK:
                predictions_json = response.json()["predictions"]
                self._faces = len(predictions_json)
                self._matched = get_matched_faces(predictions_json)

        else:
            self._faces = None
            self._matched = {}

    @property
    def attributes(self):
        """Return the classifier attributes."""
        return {
            'faces': self._faces,
            'matched_faces': self._matched,
            'total_matched_faces': len(self._matched),
        }

## Usage

In [7]:
dsface = DeepstackFace(IP_ADDRESS, PORT)

In [8]:
dsface.attributes

{'faces': None, 'matched_faces': {}, 'total_matched_faces': 0}

In [9]:
image_path = 'test-image2.jpg'

with open(image_path, "rb") as image_bytes:
    dsface.process_image_bytes(image_bytes)

In [10]:
dsface.attributes

{'faces': 2, 'matched_faces': {}, 'total_matched_faces': 0}